In [1]:
# =============================================================================
#
# --- QUBO Model Constructor for Portfolio Optimization ---
#
# Description:
#   This script builds a QUBO (Quadratic Unconstrained Binary Optimization)
#   model for a portfolio optimization problem. It loads bond data, defines
#   financial constraints (risk targeting and portfolio size), and constructs
#   the corresponding QUBO matrix `Q` and offset.
#
#   The resulting model is saved to text files, ready to be used by a
#   classical or quantum solver.
#
# Required Files:
#   - assets.csv: The raw data for all bonds.
#
# Main Outputs:
#   - qubo_matrix.txt: The generated QUBO matrix.
#   - asset_list.txt: The list of bond ISINs corresponding to the QUBO variables.
#   - qubo_offset.txt: The constant offset for the QUBO objective function.
#
# =============================================================================

import numpy as np
import pandas as pd
import random
import json

print("--- Building QUBO Model from Asset Data ---")

# --- 1. Load Data and Define Parameters ---
# --------------------------------------------
try:
    df = pd.read_csv('assets.csv')
    print("Successfully loaded 'assets.csv'.")
except FileNotFoundError:
    print("\nError: Could not find 'assets.csv'. Please make sure the file is in the same directory as this script.")
    exit()

# Define the size of the problem (number of assets to consider)
subset_size = 31
securities_df = df.head(subset_size).copy()
securities_df = securities_df.set_index('isin')
asset_list = securities_df.index.tolist()
num_assets = len(asset_list)
print(f"Building model for the first {num_assets} assets.")

# Define core optimization parameters
N = 10  # Target number of bonds in the final portfolio
characteristic_col = 'spreadDur' # The financial metric to target
betas = securities_df[characteristic_col]
lambda_penalty = 1.0  # Penalty coefficient for the size constraint

# Define risk buckets by grouping assets based on their credit quality
risk_bucket_col = 'security.elements.creditQualityBuckets'
buckets = securities_df.groupby(risk_bucket_col)
# The target for each bucket is the average of the characteristic for assets in that bucket
bucket_targets = {name: group[characteristic_col].mean() for name, group in buckets}

# --- 2. Manually Construct the QUBO Matrix (Q) and Offset ---
# ------------------------------------------------------------
# This section translates the financial rules into a mathematical QUBO form.
# ------------------------------------------------------------
Q = np.zeros((num_assets, num_assets))
offset = 0.0

# Add contributions from the Risk Bucket Hamiltonians (penalizes deviation from target)
print("Constructing QUBO matrix from risk bucket constraints...")
for name, group in buckets:
    K_target_lj = bucket_targets[name]
    bucket_isins = group.index.tolist()
    offset += K_target_lj**2
    for i in range(num_assets):
        if asset_list[i] not in bucket_isins: continue
        beta_i = betas[asset_list[i]]
        # Add linear term (diagonal of Q)
        Q[i, i] += beta_i**2 - 2 * beta_i * K_target_lj
        for j in range(i + 1, num_assets):
            if asset_list[j] not in bucket_isins: continue
            beta_j = betas[asset_list[j]]
            # Add quadratic term (off-diagonal of Q)
            Q[i, j] += 2 * beta_i * beta_j

# Add contributions from the Global Constraint Hamiltonian (penalizes incorrect portfolio size)
print("Adding global portfolio size constraint to QUBO matrix...")
offset += lambda_penalty * (N**2)
for i in range(num_assets):
    Q[i, i] += lambda_penalty * (1 - 2 * N)
    for j in range(i + 1, num_assets):
        Q[i, j] += lambda_penalty * 2

# Symmetrize the matrix, which is standard for QUBO representations
Q = (Q + Q.T) / 2

# --- 3. Save Artifacts to Text-Based Files ---
# ---------------------------------------------
# Save the complete model to robust, comma-delimited text files.
# ---------------------------------------------
np.savetxt('qubo_matrix.txt', Q, delimiter=',', fmt='%.18e')
with open('asset_list.txt', 'w') as f:
    for isin in asset_list:
        f.write(f"{isin}\n")
with open('qubo_offset.txt', 'w') as f:
    f.write(str(offset))

print("\nQUBO model files have been successfully generated:")
print("  - qubo_matrix.txt")
print("  - asset_list.txt")
print("  - qubo_offset.txt")

--- Building QUBO Model from Asset Data ---
Successfully loaded 'assets.csv'.
Building model for the first 31 assets.
Constructing QUBO matrix from risk bucket constraints...
Adding global portfolio size constraint to QUBO matrix...

QUBO model files have been successfully generated:
  - qubo_matrix.txt
  - asset_list.txt
  - qubo_offset.txt
